In [ ]:
##install and import necessary modules
##this code was originally designed and run in google colab
##use outside of colab may require modification
##if using colab, you may need to restart your runtime after installing modules,
##depending on enviornment at time of code running.

!pip install scikit-learn==1.5.2
!pip install tensorflow==2.12.1
!pip install xgboost==2.0.2
!pip install shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
import shap
import seaborn as sn
import sys
import sklearn
from google.colab import drive
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.utils import resample
from scipy.stats import mannwhitneyu
from IPython import display
from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_recall_curve, recall_score, confusion_matrix, brier_score_loss, f1_score

sn.set(style='whitegrid')
pd.set_option('display.max_columns', None)

print("Python version:", sys.version)
print("scikit-learn version:", sklearn.__version__)
print("XGBoost version:", xgb.__version__)
print("shap version:", shap.__version__)

In [ ]:
#import your dataset
##mount google drive if using in colab. Replace <MOUNT_POINT> with the directory where you want to mount the drive (e.g., /content/drive).
drive.mount('<MOUNT_POINT>')

# Replace <YOUR_FILE_PATH> with the actual path inside your Google Drive (e.g., My Drive/FileNameHere).
file_path = '<MOUNT_POINT>/<YOUR_FILE_PATH>.csv'

In [ ]:
##specify columns to load from your dataset.  We will only load the columns necessary for the score and the variables necessary for the ISS/TRISS comparisons

columns_to_load = ['AGEYEARS', 'TOTALGCS', 'SBP'
                  , 'TEMPERATURE'
                  , 'PULSERATE', 'TRISS', 'TRISS_Death', 'MORTALITY', 'TRAUMATYPE'
                  , 'WEIGHT'
                  , 'ISS_05', 'NumberOfInjuries',
                   'IntracranialVascularInjury','BrainStemInjury','EDH','SAH','SDH','SkullFx','DAI','NeckVascularInjury','ThoracicVascularInjury','AeroDigestiveInjury',
                   'CardiacInjury','LungInjury','AbdominalVascular','RibFx','KidneyInjury','StomachInjury','SpleenInjury','UroGenInternalInjury','SCI','SpineFx',
                   'UEAmputation','UEVascularInjury','UELongBoneFx','LEVascularInjury','PelvicFx','LEAmputation','PancreasInjury','LELongBoneFx','LiverInjury',
                   'ColorectalInjury','SmallBowelInjury','IPH'
                   ]

In [ ]:
# Import data and specify missing values
data = pd.read_csv(file_path, na_values=['NA', 'N/A', 'NULL', ' ', '', '-99', '-98', '-99.0', '-99.00', '-98.0', '-98.00', 'NaN'], usecols=columns_to_load)


# Filter out rows where 'TRAUMATYPE' is 26, 'Other/unspecified', or 'Burn'
try:
  exclude_values = ['26', 'Other/unspecified', 'Burn']
  data = data[~data['TRAUMATYPE'].isin(exclude_values)]
except:
  pass

##explicitly list variables that need to be present for inclusion and drop cases without these
##we cannot compare our score to ISS/TRISS without those metrics, and we need our target outcome mortality
required_vars = ['ISS_05', 'TRISS_Death', 'MORTALITY']
data = data.dropna(subset=required_vars)

# Create ShockIndex with the required logic
data['ShockIndex'] = np.where(
    data['SBP'] == 0, 2.0,  # Case where SBP is 0 → set ShockIndex to 2.0
    data['PULSERATE'] / data['SBP']  # Normal calculation
)

# Set ShockIndex to NaN if PULSERATE or SBP is missing
data.loc[data['PULSERATE'].isna() | data['SBP'].isna(), 'ShockIndex'] = np.nan

##reset indices of the df
data.reset_index(drop=True, inplace=True)

In [ ]:
##verify data appears as intended
data.head()

In [ ]:
##check for missing values
data.isnull().sum(axis=0)

In [ ]:
##create a datafram of all complications/vars to remove later.  We can remove all of these from the X data set and pick one to be
#our Y dataset

complications_df=pd.DataFrame()
complications_list= [
                    'MORTALITY', 'TRISS'
                    ]
for c in complications_list:
    complications_df[c] = data[c]
complications_df

In [ ]:
##this is where we choose our outcome variable, mortality, and give it its own dataframe

Y_data = pd.DataFrame()
Y_data['MORTALITY'] = data['MORTALITY']
Y_data

In [ ]:
##clean Y_data by replacing "Yes" and "No" vcalues with 0's and 1's

Y_data['MORTALITY'] = Y_data['MORTALITY'].replace({'Yes': 1, 'No': 0})
Y_data

In [ ]:
##now drop the outcome from our feature space as well as TRISS, since were using 1-TRISS (aka TRISS_Death) and this varibale is now useless
X_data = data.drop(columns=['MORTALITY', 'TRISS'])
X_data.shape

In [ ]:
##ensure no missing outcome data
Missing_Y = Y_data.isnull().sum(axis=0)
Missing_Y

In [ ]:
##If we have no missing values here, our data is clean
Y_clean=Y_data.copy()

In [ ]:
##if above check passes, outcome data is now clean
Missing_Y_clean = Y_clean.isnull().sum(axis=0)
Missing_Y_clean

In [ ]:
##check which variables in the input space have missing variables

Missing = X_data.isnull().sum(axis=0)
Missing[Missing>0]

In [ ]:
##order variables with missing data by percentage

data_missing = (X_data.isnull().sum(axis=0)/X_data.shape[0]) * 100
data_missing

In [ ]:
##display variables withOUT mising data

data_missing[data_missing == 0].index

In [ ]:
#remove the good columns (no missing values) from data_missing

data_missing = data_missing.drop(data_missing[data_missing == 0].index)
data_missing

In [ ]:
#sort this in ascending order
data_missing = data_missing.sort_values(ascending=False)
data_missing

In [ ]:
##prepare to drop variables with >50% missing values
##tried different cutoffs for this (33%, 66%), but 50% yielded best results

dropCutoff=50
bad_column_names = data_missing[data_missing >=dropCutoff].index
bad_column_names

In [ ]:
##actually drop bad variables
X_data_new=X_data.drop(columns=bad_column_names, axis=1)

##check for which variables still have missing data (<50% missing values)
Missing = X_data_new.isnull().sum(axis=0)
Missing[Missing>0]

In [ ]:
#display columns with less than 50% missing that need to be cleaned

to_be_cleaned_column_names = data_missing[data_missing <50].index
to_be_cleaned_column_names

In [ ]:
# Rename the 'TRAUMATYPE' column to 'Penetrating' and map the values to 0 and 1
X_data_new['Penetrating'] = X_data_new['TRAUMATYPE'].map({'Penetrating': 1, 'Blunt': 0})

# Drop the old 'TRAUMATYPE' column
X_data_new.drop(columns=['TRAUMATYPE'], inplace=True)

print(X_data_new.head())

In [ ]:
# Display the entire DataFrame without truncation
pd.set_option('display.max_columns', None)

# Get column names and data types
columns_info = []
for column_name, dtype in zip(X_data_new.columns, X_data_new.dtypes):
    columns_info.append(f"{column_name}: {dtype}")

formatted_columns_info = "\n".join(columns_info)

# Print column names and data types
print("Column Names and Data Types:")
print(formatted_columns_info)

In [ ]:
##convert No's and Yes's to 0's and 1's to minimize the amount of double variables (want to avoid Yes/Nos being converted to 1-hot variables)

try:
    X_data_new= X_data_new.replace({True: 1, 'Yes': 1, "Female": 1, False: 0, 'No': 0, "Male": 0})
except:
    pass

##drop any non blunt/penetrating mechanisms
try:
    X_data_new=X_data_new.drop(['TRAUMATYPE_26', 'TRAUMATYPE_Other/unspecified'], axis=1)
except:
    pass

X_data_new.head()

In [ ]:
##split into train, test, calibrate sets
X_train, X_test, Y_train, Y_test = train_test_split(X_data_new, Y_clean, test_size=0.2, random_state=0, stratify=Y_clean)
X_train_cal, X_val_cal, Y_train_cal, Y_val_cal = train_test_split(X_train, Y_train, test_size=0.2, random_state=0, stratify=Y_train)

In [ ]:
##perform median/mode imputation on the inputs vars that are missing
for c in to_be_cleaned_column_names:
    v = X_train[c]
    v_valid = v[~v.isnull()]

    if v.dtype == np.dtype('O'):  # Categorical column
        mode_value = v_valid.value_counts().index[0]
        for df in [X_train, X_test, X_train_cal, X_val_cal]:
            df[c] = df[c].fillna(mode_value).astype(object)

    else:  # Numeric column
        median_value = v_valid.median()
        for df in [X_train, X_test, X_train_cal, X_val_cal]:
            df[c] = df[c].fillna(median_value)


In [ ]:
##now for one-hot encoding

# Identify categorical columns from X_train only
categorical_column = [c for c in X_train_cal.columns if X_train_cal[c].dtype == np.dtype('O')]

# Apply pd.get_dummies to training data
X_train_cal = pd.get_dummies(X_train_cal, columns=categorical_column, sparse=False)

categorical_column

In [ ]:
# Align test and validation sets to match training set columns
X_test = pd.get_dummies(X_test, columns=categorical_column, sparse=False)
X_val_cal = pd.get_dummies(X_val_cal, columns=categorical_column, sparse=False)

# Ensure same columns across all datasets
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)
X_val_cal = X_val_cal.reindex(columns=X_train.columns, fill_value=0)

In [ ]:
#verify data appears as intended
X_train_cal.head()

In [ ]:
##verify no missing data in any split dataset
print(X_train_cal.isnull().sum().sum())
print(X_test.isnull().sum().sum())
print(X_val_cal.isnull().sum().sum())

In [ ]:
##final list of training columns
X_train_cal.columns

In [ ]:
#verify data is intended size
X_test.shape

In [ ]:
##now with data cleaned, take comparison vars and move them to their own dataframe prior to dropping
new_to_drop = ['TRISS_Death', 'ISS_05']

X_ISS=pd.DataFrame()
X_ISS['ISS']=X_test['ISS_05']

X_TRISS=pd.DataFrame()
X_TRISS['TRISS']=X_test['TRISS_Death']

In [ ]:
##now drop those comparison vars from the data that will be fed to the model
X_train_cal.drop(columns=new_to_drop, inplace=True)
X_test.drop(columns=new_to_drop, inplace=True)
X_val_cal.drop(columns=new_to_drop, inplace=True)


##store copies of data as tensors
X_train_tensor=X_train_cal.copy()
Y_train_tensor=Y_train_cal.copy()

X_val_tensor=X_val_cal.copy()
Y_val_tensor=Y_val_cal.copy()

X_test_tensor=X_test.copy()
Y_test_tensor=Y_test.copy()

In [ ]:
##verify data appears as intended
X_test.head()

In [ ]:
##Next step is to normalize data

scaler=StandardScaler()
#get the parameters of the transform
scaler.fit(X_train_cal)

#normalize the features in the training set
X_train_s_cal = scaler.transform(X_train_cal)
#normalize the features in the test set
print("After train/test split, X_test shape:", X_test.shape)
X_test_s = scaler.transform(X_test)
print("After scaling, X_test_s shape:", X_test_s.shape)
#normalize the features in the val set
X_val_s_cal = scaler.transform(X_val_cal)

In [ ]:
##now, fit model with hyperparameters based on other Jupyternotebook optimization
model_best_gb = xgb.XGBClassifier(random_state=0, colsample_bytree=0.6, learning_rate=0.1, max_depth=7, n_estimators=200, subsample=1.0)
model_best_gb.fit(X_train_s_cal, Y_train_cal)

In [ ]:
# Get predicted probabilities for test set (evaluate model)
from sklearn.metrics import roc_curve, auc, precision_recall_curve, recall_score, confusion_matrix

y_prob_gbo_mtp = model_best_gb.predict_proba(X_test_s)[:, 1]

# Compute AUROC on test set
auroc_gbo = roc_auc_score(Y_test, y_prob_gbo_mtp)
print(f"AUROC on the test set: {auroc_gbo}")

In [ ]:
# Calibrate the model on the validation set
calibrated_model = CalibratedClassifierCV(estimator=model_best_gb, method='isotonic', cv='prefit')
calibrated_model.fit(X_val_s_cal, Y_val_cal)

In [ ]:
# Get predicted probabilities for test set (evaluate model)
y_prob_gbo_mtp = calibrated_model.predict_proba(X_test_s)[:, 1]

# Compute AUROC on test set (calibrated)
auroc_gbo = roc_auc_score(Y_test, y_prob_gbo_mtp)
print(f"AUROC on the test set: {auroc_gbo}")

In [ ]:
##ensure true labels and predicted probabilities are in flat, clean pandas Series with aligned indices to allow for descriptive stats
y_true_series = pd.Series(Y_test.squeeze()).reset_index(drop=True)
y_prob_series = pd.Series(y_prob_gbo_mtp).reset_index(drop=True)

##get descriptive stats for population MLISS scoers
mean_prob_all = y_prob_series.mean()
median_prob_all = y_prob_series.median()
mean_prob_label1 = y_prob_series[y_true_series == 1].mean()
mean_prob_label0 = y_prob_series[y_true_series == 0].mean()
median_prob_label1 = y_prob_series[y_true_series == 1].median()
median_prob_label0 = y_prob_series[y_true_series == 0].median()

print(f"Mean predicted probability (all): {mean_prob_all:.4f}")
print(f"Mean predicted probability (label=1): {mean_prob_label1:.4f}")
print(f"Mean predicted probability (label=0): {mean_prob_label0:.4f}")

print(f"Median predicted probability (all): {median_prob_all:.4f}")
print(f"Median predicted probability (label=1): {median_prob_label1:.4f}")
print(f"Median predicted probability (label=0): {median_prob_label0:.4f}")


In [ ]:
# create a DF of true labels and predicted mortalities for more descriptive stats
df = pd.DataFrame({
    'y_true': Y_test['MORTALITY'],
    'y_prob': y_prob_gbo_mtp
})

# Q1 and Q3 for overall cohort
q1_all = df['y_prob'].quantile(0.25)
q3_all = df['y_prob'].quantile(0.75)

# Q1 and Q3 for label = 1
q1_label1 = df[df['y_true'] == 1]['y_prob'].quantile(0.25)
q3_label1 = df[df['y_true'] == 1]['y_prob'].quantile(0.75)

# Q1 and Q3 for label = 0
q1_label0 = df[df['y_true'] == 0]['y_prob'].quantile(0.25)
q3_label0 = df[df['y_true'] == 0]['y_prob'].quantile(0.75)

# Print results
print(f"Overall:     Q1 = {q1_all:.4f}, Q3 = {q3_all:.4f}")
print(f"Label = 1:   Q1 = {q1_label1:.4f}, Q3 = {q3_label1:.4f}")
print(f"Label = 0:   Q1 = {q1_label0:.4f}, Q3 = {q3_label0:.4f}")

In [ ]:
##evaluate whether MLISS score distributions and differnet between surivvors and deceased patients with Mann Whitney U

# Split into two groups
probs_label1 = df[df['y_true'] == 1]['y_prob']
probs_label0 = df[df['y_true'] == 0]['y_prob']

# Perform Mann-Whitney U test
u_statistic, p_value = mannwhitneyu(probs_label1, probs_label0, alternative='two-sided')

print(f"Mann-Whitney U statistic: {u_statistic:.4f}")
print(f"P-value: {p_value:.4f}")

In [ ]:
# Get predicted probabilities for test set (evaluate model)
y_prob_gbo = calibrated_model.predict_proba(X_test_s)[:, 1]

# Compute AUROC on test set (calibrated)
auroc_gbo = roc_auc_score(Y_test, y_prob_gbo)
print(f"AUROC on the test set: {auroc_gbo}")


# Calculate ROC curve
y_pred_prob_gbo = calibrated_model.predict_proba(X_test_s)[:, 1]
fpr_gbo, tpr_gbo, thresholds = roc_curve(Y_test, y_pred_prob_gbo)

# Calculate the Area Under the ROC Curve (AUC)
roc_auc_gbo = auc(fpr_gbo, tpr_gbo)

# Plot ROC curve
plt.figure(figsize=(8, 8))
plt.plot(fpr_gbo, tpr_gbo, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc_gbo:.3f})')
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

# Calculate precision and recall values
precision, recall, thresholds = precision_recall_curve(Y_test, y_prob_gbo_mtp)

# Calculate area under the precision-recall curve
pr_auc = auc(recall, precision)

# Plot the precision-recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='blue', label=f'PR curve (AUC={pr_auc:.3f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='best')
plt.show()

# Choose a custom threshold (e.g., 0.3)
##this value yielded the highest value for accuracy
custom_threshold = 0.5

# Apply the custom threshold to make binary predictions
y_pred_custom_threshold = (y_prob_gbo > custom_threshold).astype(int)

num_positive_cases = sum(y_pred_custom_threshold)
num_negative_cases = len(y_pred_custom_threshold) - num_positive_cases
print("Number of Cases Classified as Positive:", num_positive_cases)
print("Number of Cases Classified as Negative:", num_negative_cases)


# Calculate confusion matrix
conf_matrix = confusion_matrix(Y_test, y_pred_custom_threshold)

print('CONFUSION MATRIX')
print('-----------')
print('|',conf_matrix[0,0],'|', conf_matrix[0,1], '|')
print('-----------')
print('|',conf_matrix[1,0],'|', conf_matrix[1,1], '|')

# Calculate false positive rate (FPR) and false negative rate (FNR)

falseNeg = conf_matrix[1, 0]
falsePos = conf_matrix[0, 1]
trueNeg = conf_matrix[0, 0]
truePos = conf_matrix[1, 1]
actualPos = truePos + falseNeg
actualNeg = trueNeg + falsePos
predPos = truePos + falsePos
predNeg = trueNeg + falseNeg


fpr = falsePos / actualNeg
fnr = falseNeg / actualPos
tpr = truePos / actualPos
tnr = trueNeg / actualNeg
accuracy = (truePos + trueNeg) / (actualPos + actualNeg)
ppV = truePos / predPos
npV = trueNeg / predNeg

brier_score = brier_score_loss(Y_test, y_pred_prob_gbo)

print('Actual Positives:', actualPos)
print('Actual Negatives:', actualNeg)
print("False Positive Rate (FPR):", fpr)
print("False Negative Rate (FNR):", fnr)
print("True Positive Rate (TPR) (Recall/Sensitivity):", tpr) ##right
print("True Negative Rate (TNR) (specificity):", tnr)
print('Accuracy:', accuracy) ##right
print('Precision (PPV),', ppV) ##right
print('NPV', npV)
print("Brier Score:", brier_score)

# Convert probabilities to binary labels using the threshold
y_pred_binary10 = (y_prob_gbo > 0.1).astype(int)
y_pred_binary20 = (y_prob_gbo > 0.2).astype(int)
y_pred_binary30 = (y_prob_gbo > 0.3).astype(int)
y_pred_binary40 = (y_prob_gbo > 0.4).astype(int)
y_pred_binary50 = (y_prob_gbo > 0.5).astype(int)
y_pred_binary60 = (y_prob_gbo > 0.6).astype(int)
y_pred_binary70 = (y_prob_gbo > 0.7).astype(int)
y_pred_binary80 = (y_prob_gbo > 0.8).astype(int)
y_pred_binary90 = (y_prob_gbo > 0.9).astype(int)
y_pred_binary = (y_prob_gbo > custom_threshold).astype(int)

# Compute F1 score
f1 = f1_score(Y_test, y_pred_binary)
f1_10 = f1_score(Y_test, y_pred_binary10)
f1_20 = f1_score(Y_test, y_pred_binary20)
f1_30 = f1_score(Y_test, y_pred_binary30)
f1_40 = f1_score(Y_test, y_pred_binary40)
f1_50 = f1_score(Y_test, y_pred_binary50)
f1_60 = f1_score(Y_test, y_pred_binary60)
f1_70 = f1_score(Y_test, y_pred_binary70)
f1_80 = f1_score(Y_test, y_pred_binary80)
f1_90 = f1_score(Y_test, y_pred_binary90)

print("F1 Score:", f1)
print("F1-10 Score:", f1_10)
print("F1-20 Score:", f1_20)
print("F1-30 Score:", f1_30)
print("F1-40 Score:", f1_40)
print("F1-50 Score:", f1_50)
print("F1-60 Score:", f1_60)
print("F1-70 Score:", f1_70)
print("F1-80 Score:", f1_80)
print("F1-90 Score:", f1_90)

In [ ]:
##evaluate TRISS as predictive tool
predicted_prob_triss = X_TRISS['TRISS']  # This should be probabilities (e.g., 0.8, 0.3)
true_label = Y_test['MORTALITY']          # This should be binary labels (e.g., 1 for death, 0 for survival)

# Calculate FPR, TPR, and thresholds
fpr_triss, tpr_triss, thresholds_triss = roc_curve(true_label, predicted_prob_triss)

# Calculate the area under the ROC curve
roc_auc_triss = auc(fpr_triss, tpr_triss)

# Plot the ROC curve
plt.figure(figsize=(8, 8))
plt.plot(fpr_triss, tpr_triss, color='darkorange', lw=2, label=f'ROC AUC = {roc_auc_triss:.3f}')
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--',label='Random')  # Dashed diagonal line for random model
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()



# Print the ROC AUC value
print(f'ROC AUC: {roc_auc_triss:.3f}')

In [ ]:
# Fit logistic regression to map ISS scores to probabilities
lr_iss = LogisticRegression()
lr_iss.fit(X_ISS.values.reshape(-1,1), Y_test)

# Predict probabilities
iss_probs = lr_iss.predict_proba(X_ISS.values.reshape(-1,1))[:,1]

In [ ]:
##evaluate ISS as predictive tool

predicted_prob_iss = iss_probs
true_label = Y_test['MORTALITY']   # This should be the binary outcome variable (0 or 1)

# Calculate the FPR, TPR, and thresholds
fpr_iss, tpr_iss, thresholds_iss = roc_curve(true_label, predicted_prob_iss)

# Calculate the area under the ROC curve (AUROC)
roc_auc_iss = auc(fpr_iss, tpr_iss)

# Plot the ROC curve
plt.figure(figsize=(8, 8))
plt.plot(fpr_triss, tpr_triss, color='darkorange', lw=2, label=f'ROC AUC = {roc_auc_iss:.3f}')
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--',label='Random')  # Dashed diagonal line for random model
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

# Print the AUROC value
print(f'ROC AUC: {roc_auc_iss:.3f}')

In [ ]:
# Plot ROC curves of all 3
plt.figure(figsize=(8, 8))
plt.plot(fpr_gbo, tpr_gbo, color='b', lw=2, label=f'ROC curve (area = {roc_auc_gbo:.3f})')
plt.plot(fpr_triss, tpr_triss, color='green', lw=2, label=f'ROC AUC TRISS = {roc_auc_triss:.3f}')
plt.plot(fpr_iss, tpr_iss, color='darkorange', lw=2, label=f'ROC AUC ISS = {roc_auc_iss:.3f}')
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')

In [ ]:
##now we evaluate calibration of all 3 metrics (MLISS, TRISS, ISS)

def bootstrap_calibration_curve(y_true, y_prob, n_bins=10, n_boot=1000, random_state=None):
    """
    1) Compute the original bin-based calibration curve.
    2) Bootstrap the dataset n_boot times, each time recalculating the bin-based
       fraction of positives (prob_true) and storing it.
    3) Return the original curve + 95% CI per bin (based on 2.5 and 97.5 percentiles).
    """
    # -----------------------------
    # Original calibration curve
    # -----------------------------
    # prob_true_orig, prob_pred_orig = calibration_curve(...) does binning internally.
    # But we want to fix n_bins and ensure consistent binning across bootstraps.
    # We'll do a manual binning approach here to keep consistent bin boundaries.

    # Define bin edges (equally spaced from 0 to 1)
    bin_edges = np.linspace(0, 1, n_bins + 1)
    # Digitize predicted probabilities
    bin_indices = np.digitize(y_prob, bin_edges) - 1
    bin_indices[bin_indices == n_bins] = n_bins - 1  # cap any == n_bins to last bin

    # Prepare arrays to hold the original bin stats
    prob_pred_orig = np.zeros(n_bins)
    prob_true_orig = np.zeros(n_bins)
    counts_in_bin = np.zeros(n_bins, dtype=int)

    # Fill in the stats for each bin
    for i in range(n_bins):
        mask = (bin_indices == i)
        counts_in_bin[i] = np.sum(mask)
        if counts_in_bin[i] > 0:
            prob_pred_orig[i] = np.mean(y_prob[mask])   # mean predicted prob in this bin
            prob_true_orig[i] = np.mean(y_true[mask])   # fraction of positives (actual)
        else:
            # If bin is empty, set to NaN
            prob_pred_orig[i] = np.nan
            prob_true_orig[i] = np.nan

    # Remove empty bins (NaN) from the original arrays
    valid_mask = ~np.isnan(prob_pred_orig)
    prob_pred_orig = prob_pred_orig[valid_mask]
    prob_true_orig = prob_true_orig[valid_mask]

    # -----------------------------
    # Bootstrap to get CIs
    # -----------------------------
    rng = np.random.RandomState(random_state) if random_state else np.random

    # We'll store the fraction of positives (prob_true) for each bin in each bootstrap
    # but only for the bins that were valid in the original data
    boot_prob_true = np.zeros((n_boot, sum(valid_mask)))

    n_data = len(y_true)
    data_idx = np.arange(n_data)

    for b in range(n_boot):
        # Sample with replacement
        sample_indices = rng.randint(0, n_data, size=n_data)
        y_true_b = y_true[sample_indices]
        y_prob_b = y_prob[sample_indices]

        # Repeat the binning steps
        bin_indices_b = np.digitize(y_prob_b, bin_edges) - 1
        bin_indices_b[bin_indices_b == n_bins] = n_bins - 1

        prob_true_b = np.zeros(n_bins)
        for i in range(n_bins):
            mask_b = (bin_indices_b == i)
            if np.sum(mask_b) > 0:
                prob_true_b[i] = np.mean(y_true_b[mask_b])
            else:
                prob_true_b[i] = np.nan

        # filter to only valid bins
        prob_true_b = prob_true_b[valid_mask]
        boot_prob_true[b, :] = prob_true_b

    # Compute 2.5th and 97.5th percentile per bin (column-wise)
    lower_ci = np.nanpercentile(boot_prob_true, 2.5, axis=0)
    upper_ci = np.nanpercentile(boot_prob_true, 97.5, axis=0)

    return prob_pred_orig, prob_true_orig, lower_ci, upper_ci

# -----------------------------------------------------------
# Now apply the function
# -----------------------------------------------------------

# === Input arrays ===
y_true = np.array(Y_test)
y_prob_gbo = np.array(y_prob_gbo)       # ML model
y_prob_triss = np.array(X_TRISS['TRISS'])  # or however you're storing TRISS scores
y_prob_iss = np.array(iss_probs)        # or however you're storing ISS scores

# === Helper function to compute sorted calibration data + CIs ===
def get_bootstrap_calibration_data(y_true, y_prob, label, color, n_bins=10, n_boot=1000, random_state=42):
    prob_pred, prob_true, lower_ci, upper_ci = bootstrap_calibration_curve(
        y_true, y_prob, n_bins=n_bins, n_boot=n_boot, random_state=random_state
    )
    sort_idx = np.argsort(prob_pred)
    return {
        "x": prob_pred[sort_idx],
        "y": prob_true[sort_idx],
        "lower": lower_ci[sort_idx],
        "upper": upper_ci[sort_idx],
        "label": label,
        "color": color
    }

# === Get calibration data ===
calib_gbo = get_bootstrap_calibration_data(y_true, y_prob_gbo, label="ML Model", color='b')
calib_triss = get_bootstrap_calibration_data(y_true, y_prob_triss, label="TRISS", color='g')
calib_iss = get_bootstrap_calibration_data(y_true, y_prob_iss, label="ISS", color='darkorange')

# === Compute Brier scores ===
brier_gbo = brier_score_loss(y_true, y_prob_gbo)
brier_triss = brier_score_loss(y_true, y_prob_triss)
brier_iss = brier_score_loss(y_true, y_prob_iss)

print(f"Brier Score - ML Model: {brier_gbo:.4f}")
print(f"Brier Score - TRISS:    {brier_triss:.4f}")
print(f"Brier Score - ISS:      {brier_iss:.4f}")

# === Plot Reliability Diagram ===
plt.figure(figsize=(8, 6))
plt.plot([0, 1], [0, 1], 'k--', label='Perfect Calibration')

for calib in [calib_gbo, calib_triss, calib_iss]:
    plt.plot(calib["x"], calib["y"], marker='o', label=calib["label"], color=calib["color"])
    plt.fill_between(calib["x"], calib["lower"], calib["upper"], color=calib["color"], alpha=0.2)

plt.xlabel('Mean Predicted Probability')
plt.ylabel('Observed Mortality Rate')
plt.title('Reliability Diagram with 95% CI – All Methods')
plt.legend(loc='best')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.grid(True)
plt.show()

In [ ]:
# Decision Curve analysis
# ========================================

##first define function to calculate net benefit
def net_benefit(y_true, y_prob, thresholds):
    """
    NetBenefit = (TP/N) - (FP/N)*(threshold/(1-threshold))
    """
    N = len(y_true)
    NB = []
    for t in thresholds:
        y_pred = (y_prob >= t).astype(int)
        TP = np.sum((y_true == 1) & (y_pred == 1))
        FP = np.sum((y_true == 0) & (y_pred == 1))
        if t == 1.0:
            nb_t = 0
        else:
            nb_t = (TP / N) - (FP / N) * (t / (1 - t))
        NB.append(nb_t)
    return NB

# Thresholds for decision curve
decision_thresholds = np.linspace(0.0, 1.0, 101)

# Make sure y_true is an array
y_true_array = np.asarray(Y_test).flatten()

# Net benefit for MLISS
NB_model = net_benefit(y_true_array, y_prob_gbo, decision_thresholds)

# # Net benefit for ISS (ISS predictions: X_ISS.values)
NB_ISS = net_benefit(y_true_array, iss_probs, decision_thresholds)

# Net benefit for TRISS (TRISS predictions: X_TRISS.values)
NB_TRISS = net_benefit(y_true_array, X_TRISS.values.flatten(), decision_thresholds)

# Net benefit for treat all and treat none (baseline strategies)
N = len(Y_test)
prevalence = np.mean(Y_test)  # fraction of positives
treat_all_nb = []
for t in decision_thresholds:
    if t == 1.0:
        treat_all_nb.append(0)
    else:
        treat_all_nb.append(prevalence - (1 - prevalence)*(t/(1-t)))

treat_none_nb = np.zeros_like(decision_thresholds)

# now plot DCA
plt.figure(figsize=(8, 6))
plt.plot(decision_thresholds, NB_model, label='New ML Model', color='b')
plt.plot(decision_thresholds, NB_ISS, label='ISS', color='darkorange')
plt.plot(decision_thresholds, NB_TRISS, label='TRISS', color='g')
plt.plot(decision_thresholds, treat_all_nb, label='Treat All', color='red', linestyle='--')
plt.plot(decision_thresholds, treat_none_nb, label='Treat None', color='grey', linestyle=':')

plt.xlabel('Threshold Probability')
plt.ylabel('Net Benefit')
plt.title('Decision Curve Analysis')
plt.legend(loc='best')
plt.ylim([-0.04, 0.04])
plt.xlim([0, 1.0])
plt.grid(True)
plt.show()

In [ ]:
# ========================================
#  Confusion Matrix Stats @ threshold=0.5, similar to above derivation
# ========================================
custom_threshold = 0.5
y_pred_custom_threshold = (y_prob_gbo >= custom_threshold).astype(int)

conf_matrix = confusion_matrix(Y_test, y_pred_custom_threshold)
falseNeg, falsePos, trueNeg, truePos = conf_matrix[1, 0], conf_matrix[0, 1], conf_matrix[0, 0], conf_matrix[1, 1]
actualPos, actualNeg = truePos + falseNeg, trueNeg + falsePos
predPos, predNeg = truePos + falsePos, trueNeg + falseNeg

fpr = falsePos / actualNeg if actualNeg else 0
fnr = falseNeg / actualPos if actualPos else 0
tpr = truePos / actualPos if actualPos else 0
tnr = trueNeg / actualNeg if actualNeg else 0
accuracy = (truePos + trueNeg) / (actualPos + actualNeg) if (actualPos + actualNeg) else 0
ppV = truePos / predPos if predPos != 0 else 0
npV = trueNeg / predNeg if predNeg != 0 else 0

brier_score = brier_score_loss(Y_test, y_prob_gbo)

print("\n=== Threshold = 0.50 ===")
print("Confusion Matrix:")
print(conf_matrix)
print(f"False Positive Rate (FPR): {fpr:.3f}")
print(f"False Negative Rate (FNR): {fnr:.3f}")
print(f"True Positive Rate (TPR/Recall): {tpr:.3f}")
print(f"True Negative Rate (TNR/Specificity): {tnr:.3f}")
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision (PPV): {ppV:.3f}")
print(f"Negative Predictive Value (NPV): {npV:.3f}")
print(f"Brier Score: {brier_score:.4f}")

# ========================================
# Find Threshold that Maximizes F1 (0.01 increments)
# ========================================
candidate_thresholds = np.linspace(0, 1, 101)  # from 0.00 to 1.00 in 0.01 steps
best_f1 = -1.0
best_f1_threshold = 0.0

for t in candidate_thresholds:
    y_pred = (y_prob_gbo >= t).astype(int)
    f1_val = f1_score(Y_test, y_pred)
    if f1_val > best_f1:
        best_f1 = f1_val
        best_f1_threshold = t

print(f"\nOptimal threshold for max F1 (nearest hundredth): {best_f1_threshold:.2f}")
print(f"Max F1 Score at that threshold: {best_f1:.3f}")

# ========================================
# Find Threshold for >=90% Specificity (0.01 increments)
# ========================================
tnr_90_thresh = None

for t in candidate_thresholds:
    y_pred = (y_prob_gbo >= t).astype(int)
    cm = confusion_matrix(Y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    actual_neg = tn + fp
    if actual_neg == 0:
        continue

    specificity = tn / actual_neg  # TNR
    if specificity >= 0.90:
        tnr_90_thresh = t
        break  # first threshold that achieves TNR >= 0.90

if tnr_90_thresh is not None:
    print(f"\nThreshold achieving >=90% specificity (nearest hundredth): {tnr_90_thresh:.2f}")
    # Recompute confusion matrix & TPR at that threshold
    y_pred_90sp = (y_prob_gbo >= tnr_90_thresh).astype(int)
    cm_90 = confusion_matrix(Y_test, y_pred_90sp)
    tn_90, fp_90, fn_90, tp_90 = cm_90.ravel()
    tnr_val_90 = tn_90 / (tn_90 + fp_90) if (tn_90 + fp_90) else 0
    tpr_val_90 = tp_90 / (tp_90 + fn_90) if (tp_90 + fn_90) else 0
    print(f"Specificity (TNR) at {tnr_90_thresh:.2f}: {tnr_val_90:.3f}")
    print(f"Sensitivity (TPR) at {tnr_90_thresh:.2f}: {tpr_val_90:.3f}")
else:
    print("\nNo threshold found where TNR >= 0.90 in [0.00, 1.00].")
# ========================================
# Define a Function to Compute Metrics
# ========================================
def compute_metrics(y_true, y_prob, threshold):
    """
    Computes performance metrics at a given threshold.

    Parameters:
    - y_true: True binary labels
    - y_prob: Predicted probabilities for the positive class
    - threshold: Threshold to convert probabilities to binary predictions

    Returns:
    - metrics_dict: Dictionary containing performance metrics
    """
    y_pred = (y_prob >= threshold).astype(int)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) else 0
    sensitivity = tp / (tp + fn) if (tp + fn) else 0  # TPR
    specificity = tn / (tn + fp) if (tn + fp) else 0  # TNR
    precision = tp / (tp + fp) if (tp + fp) else 0      # PPV
    npv = tn / (tn + fn) if (tn + fn) else 0            # NPV
    f1 = f1_score(y_true, y_pred) if (tp + fp + fn) else 0

    metrics_dict = {
        'Accuracy': accuracy,
        'Sensitivity (TPR)': sensitivity,
        'Specificity (TNR)': specificity,
        'Precision (PPV)': precision,
        'Negative Predictive Value (NPV)': npv,
        'F1 Score': f1
    }

    return metrics_dict

# ========================================
# Create and Print Performance Tables
# ========================================
# Initialize an empty list to store metric dictionaries
performance_metrics = []

# Thresholds to report
thresholds_to_report = {
    'Max F1 Score Threshold': best_f1_threshold,
    '90% Specificity Threshold': tnr_90_thresh
}

# Compute metrics for each threshold
for desc, thresh in thresholds_to_report.items():
    if thresh is not None:
        metrics = compute_metrics(Y_test, y_prob_gbo, thresh)
        metrics['Threshold'] = f"{thresh:.2f}"
        performance_metrics.append(metrics)
    else:
        print(f"\n{desc} is not available as no threshold achieved >=90% specificity.")

# Create a DataFrame
df_performance = pd.DataFrame(performance_metrics)

# Reorder columns for better readability
df_performance = df_performance[['Threshold', 'Accuracy', 'Sensitivity (TPR)',
                                 'Specificity (TNR)', 'Precision (PPV)',
                                 'Negative Predictive Value (NPV)', 'F1 Score']]

# Set descriptive index
df_performance.index = thresholds_to_report.keys()

# Display the tables
print("\n=== Performance Metrics at Selected Thresholds ===")
print(df_performance.to_string(float_format="{:.3f}".format))

# ISS (already probability)
y_pred_iss = (iss_probs.flatten() >= 0.27).astype(int)
f1_iss = f1_score(Y_test, y_pred_iss)

# TRISS (already probability)
y_pred_triss = (X_TRISS.values.flatten() >= 0.27).astype(int)
f1_triss = f1_score(Y_test, y_pred_triss)

# Now you have best threshold and F1 for your ML model
print("\n=== F1 Scores ===")
print(f"F1 Score (Your ML Model @ optimal threshold): {best_f1:.3f}")
print(f"F1 Score (TRISS @ 0.5 threshold): {f1_triss:.3f}")
print(f"F1 Score (ISS Probs @ 0.5 threshold): {f1_iss:.3f}")


In [ ]:
# ======================================
# Find Optimal Thresholds for Max F1 and ~90% Specificity
# ======================================

candidate_thresholds = np.linspace(0, 1, 101)  # from 0.00 to 1.00 in 0.01 steps

# Initialize trackers
best_f1_iss = -1.0
best_f1_threshold_iss = 0.0
spec90_threshold_iss = None
closest_spec90_diff_iss = float("inf")

best_f1_triss = -1.0
best_f1_threshold_triss = 0.0
spec90_threshold_triss = None
closest_spec90_diff_triss = float("inf")

# Function to compute specificity
def get_specificity(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

# 1. ISS
for t in candidate_thresholds:
    y_pred = (iss_probs.flatten() >= t).astype(int)
    f1_val = f1_score(Y_test, y_pred)
    spec_val = get_specificity(Y_test, y_pred)

    # Max F1
    if f1_val > best_f1_iss:
        best_f1_iss = f1_val
        best_f1_threshold_iss = t

    # Closest to 90% specificity
    if abs(spec_val - 0.90) < closest_spec90_diff_iss:
        closest_spec90_diff_iss = abs(spec_val - 0.90)
        spec90_threshold_iss = t

# 2. TRISS
for t in candidate_thresholds:
    y_pred = (X_TRISS.values.flatten() >= t).astype(int)
    f1_val = f1_score(Y_test, y_pred)
    spec_val = get_specificity(Y_test, y_pred)

    # Max F1
    if f1_val > best_f1_triss:
        best_f1_triss = f1_val
        best_f1_threshold_triss = t

    # Closest to 90% specificity
    if abs(spec_val - 0.90) < closest_spec90_diff_triss:
        closest_spec90_diff_triss = abs(spec_val - 0.90)
        spec90_threshold_triss = t

# ========================================
# Print Results
# ========================================
print("\n=== Optimal Thresholds ===")
print(f"ISS - Best F1 = {best_f1_iss:.3f} at Threshold = {best_f1_threshold_iss:.2f}")
print(f"ISS - ~90% Specificity Threshold = {spec90_threshold_iss:.2f}")

print(f"TRISS - Best F1 = {best_f1_triss:.3f} at Threshold = {best_f1_threshold_triss:.2f}")
print(f"TRISS - ~90% Specificity Threshold = {spec90_threshold_triss:.2f}")


In [ ]:
# ========================================
# Expanded Performance Metrics for ML, ISS, and TRISS at Both Thresholds
# ========================================

# 1. Compute performance metrics at each threshold
performance_metrics_all = []

# ML Model
for desc, thresh in thresholds_to_report.items():
    if thresh is not None:
        metrics = compute_metrics(Y_test, y_prob_gbo, thresh)
        metrics['Threshold'] = f"{thresh:.2f}"
        metrics['Model'] = 'ML Model'
        metrics['Threshold Type'] = desc
        performance_metrics_all.append(metrics)

# ISS Model at both thresholds
# Max F1
metrics_iss_f1 = compute_metrics(Y_test, iss_probs.flatten(), best_f1_threshold_iss)
metrics_iss_f1['Threshold'] = f"{best_f1_threshold_iss:.2f}"
metrics_iss_f1['Model'] = 'ISS'
metrics_iss_f1['Threshold Type'] = 'Max F1 Score Threshold'
performance_metrics_all.append(metrics_iss_f1)

# ~90% Specificity
metrics_iss_spec90 = compute_metrics(Y_test, iss_probs.flatten(), spec90_threshold_iss)
metrics_iss_spec90['Threshold'] = f"{spec90_threshold_iss:.2f}"
metrics_iss_spec90['Model'] = 'ISS'
metrics_iss_spec90['Threshold Type'] = '90% Specificity Threshold'
performance_metrics_all.append(metrics_iss_spec90)

# TRISS Model at both thresholds
# Max F1
metrics_triss_f1 = compute_metrics(Y_test, X_TRISS.values.flatten(), best_f1_threshold_triss)
metrics_triss_f1['Threshold'] = f"{best_f1_threshold_triss:.2f}"
metrics_triss_f1['Model'] = 'TRISS'
metrics_triss_f1['Threshold Type'] = 'Max F1 Score Threshold'
performance_metrics_all.append(metrics_triss_f1)

# ~90% Specificity
metrics_triss_spec90 = compute_metrics(Y_test, X_TRISS.values.flatten(), spec90_threshold_triss)
metrics_triss_spec90['Threshold'] = f"{spec90_threshold_triss:.2f}"
metrics_triss_spec90['Model'] = 'TRISS'
metrics_triss_spec90['Threshold Type'] = '90% Specificity Threshold'
performance_metrics_all.append(metrics_triss_spec90)

# ========================================
# Create and display Combined DataFrame
# ========================================
df_performance_all = pd.DataFrame(performance_metrics_all)

# Reorder columns
df_performance_all = df_performance_all[['Model', 'Threshold Type', 'Threshold', 'Accuracy',
                                         'Sensitivity (TPR)', 'Specificity (TNR)',
                                         'Precision (PPV)', 'Negative Predictive Value (NPV)',
                                         'F1 Score']]

# Display results
print("\n=== Expanded Performance Metrics ===")
print(df_performance_all.to_string(float_format="{:.3f}".format))


In [ ]:
# Brier score for ISS
brier_score_iss = brier_score_loss(Y_test, iss_probs.flatten())
print(f"Brier Score (ISS): {brier_score_iss:.4f}")

# Brier score for TRISS
brier_score_triss = brier_score_loss(Y_test, X_TRISS.values.flatten())
print(f"Brier Score (TRISS): {brier_score_triss:.4f}")

In [ ]:
##now, comput 95% confidence interval based on 1000-sample bootstrapping

# Initialize your classifier (replace with your actual classifier)
classifier = model_best_gb

# Compute the AUROC on the original test set
original_auroc = auroc_gbo



# Number of bootstrap samples
n_bootstrap_samples = 1000

# Initialize an array to store bootstrapped AUROC values
bootstrap_aurocs = np.zeros(n_bootstrap_samples)

# Perform bootstrapping
for i in range(n_bootstrap_samples):
    # Generate a bootstrap sample
    bootstrap_indices = np.random.choice(len(Y_test_tensor), len(Y_test_tensor), replace=True)
    y_bootstrap = y_prob_gbo_mtp[bootstrap_indices]
    y_true_bootstrap = Y_test_tensor.iloc[bootstrap_indices]

    # Compute AUROC on the bootstrap sample
    bootstrap_aurocs[i] = roc_auc_score(y_true_bootstrap, y_bootstrap)

# Calculate the confidence interval
confidence_interval = np.percentile(bootstrap_aurocs, [2.5, 97.5])

print("Original AUROC:", original_auroc)
print("95% Confidence Interval:", confidence_interval)

In [ ]:
# Compute AUROC ISS
auroc_iss = roc_auc_score(Y_test, X_ISS)
print(f"AUROC on the test set: {auroc_iss}")

In [ ]:
# Compute AUROC TRISS
auroc_triss = roc_auc_score(Y_test, X_TRISS)
print(f"AUROC on the test set: {auroc_triss}")

In [ ]:
##now for paired bootstrap testing between MLISS and TRISS and MLISS and ISS
##first lets define a helper function

def paired_bootstrap_auc_test(
    y_true,
    predA,
    predB,
    n_boot=1000,
    alpha=0.05,
    random_state=None
):
    """
    Compare two correlated AUCs (predA vs. predB) via paired bootstrapping,
    also returning 95% CI for each AUC individually.

    Parameters
    ----------
    y_true : array-like of shape (n_samples,)
        Ground truth binary labels (0 or 1).
    predA : array-like of shape (n_samples,)
        Scores (e.g., probabilities) from model/variable A.
    predB : array-like of shape (n_samples,)
        Scores from model/variable B.
    n_boot : int, default=1000
        Number of bootstrap iterations.
    alpha : float, default=0.05
        Significance level (for the (1 - alpha) CIs).
    random_state : int or None
        Random seed for reproducibility.

    Returns
    -------
    results : dict
        Dictionary containing:
        - aucA, aucB: AUC on the original dataset for each method.
        - aucA_ci_lower, aucA_ci_upper: (1 - alpha) CI for A's AUC.
        - aucB_ci_lower, aucB_ci_upper: (1 - alpha) CI for B's AUC.
        - baseline_diff: AUC(A) - AUC(B) on the full dataset (no resampling).
        - mean_diff: Mean difference of AUCs across bootstrap samples.
        - diff_ci_lower, diff_ci_upper: (1 - alpha) CI for AUC difference.
        - p_value: Approx. two-sided p-value for difference in AUC.
    """

    # Convert to NumPy arrays
    y_true = np.asarray(y_true)
    predA = np.asarray(predA)
    predB = np.asarray(predB)

    # Basic checks
    assert len(y_true) == len(predA) == len(predB), "Arrays must all have the same length."
    n = len(y_true)

    # Compute AUC on the full (original) dataset
    aucA = roc_auc_score(y_true, predA)
    aucB = roc_auc_score(y_true, predB)
    baseline_diff = aucA - aucB

    # Random generator
    rng = np.random.default_rng(random_state)

    # Arrays to store bootstrapped AUCs
    aucAs = np.zeros(n_boot)
    aucBs = np.zeros(n_boot)
    diffs = np.zeros(n_boot)

    for i in range(n_boot):
        # 1) Sample n indices with replacement
        sample_idx = rng.integers(0, n, size=n)

        # 2) Create bootstrap samples
        y_samp = y_true[sample_idx]
        A_samp = predA[sample_idx]
        B_samp = predB[sample_idx]

        # 3) Compute AUC for each method
        aucA_samp = roc_auc_score(y_samp, A_samp)
        aucB_samp = roc_auc_score(y_samp, B_samp)

        # 4) Store AUCs and difference
        aucAs[i] = aucA_samp
        aucBs[i] = aucB_samp
        diffs[i] = aucA_samp - aucB_samp

    # Compute individual AUC CIs
    aucA_ci_lower = np.percentile(aucAs, 100 * (alpha / 2))
    aucA_ci_upper = np.percentile(aucAs, 100 * (1 - alpha / 2))

    aucB_ci_lower = np.percentile(aucBs, 100 * (alpha / 2))
    aucB_ci_upper = np.percentile(aucBs, 100 * (1 - alpha / 2))

    # Compute difference CI
    diff_ci_lower = np.percentile(diffs, 100 * (alpha / 2))
    diff_ci_upper = np.percentile(diffs, 100 * (1 - alpha / 2))

    # Approx. two-sided p-value by sign test on "diffs"
    n_neg = np.sum(diffs < 0)
    n_pos = np.sum(diffs > 0)
    p_val = 2.0 * min(n_neg, n_pos) / n_boot
    p_val = min(p_val, 1.0)

    coverage = (1 - alpha) * 100.0  # e.g., 95.0 if alpha=0.05

    return {
        "aucA": aucA,
        "aucB": aucB,
        "aucA_ci_lower": aucA_ci_lower,
        "aucA_ci_upper": aucA_ci_upper,
        "aucB_ci_lower": aucB_ci_lower,
        "aucB_ci_upper": aucB_ci_upper,
        "baseline_diff": baseline_diff,
        "mean_diff": np.mean(diffs),
        "diff_ci_lower": diff_ci_lower,
        "diff_ci_upper": diff_ci_upper,
        "p_value": p_val,
        "coverage": coverage
    }


In [ ]:
#now actually run paired bootstrapping
# Y_test: ground truth labels (0 or 1)
# ML_preds: predictions from your ML model
# ISS_vals, pTRISS_vals, TRISS_vals: numeric scores

pairs = {
    "ISS": X_ISS.values,
    "TRISS": X_TRISS.values,
}

for var_name, var_array in pairs.items():
    results = paired_bootstrap_auc_test(
        y_true=Y_test,
        predA=y_prob_gbo_mtp,
        predB=var_array,
        n_boot=2000,      # or more for higher precision
        alpha=(0.05/2),   #for 97.5% CI (bonferroni correction for 2 comparisons)
        random_state=42
    )
    coverage_str = f"{results['coverage']:.1f}%"

    print(f"--- ML Model vs. {var_name} ---")
    print(f"AUC(ML) = {results['aucA']:.3f}, {coverage_str} CI: "
          f"[{results['aucA_ci_lower']:.3f}, {results['aucA_ci_upper']:.3f}]")
    print(f"AUC({var_name}) = {results['aucB']:.3f}, {coverage_str} CI: "
          f"[{results['aucB_ci_lower']:.3f}, {results['aucB_ci_upper']:.3f}]")
    print(f"AUC diff (ML - {var_name}) = {results['baseline_diff']:.4f}, {coverage_str} CI: "
          f"[{results['diff_ci_lower']:.4f}, {results['diff_ci_upper']:.4f}]")
    print(f"p-value = {results['p_value']:.4f}\n")



In [ ]:
##now lets get 95% cI for PR curve for MLISS

# Number of bootstrap iterations
n_iterations = 1000
# List to store bootstrapped AUPRC values
auprc_scores = []

# Original precision-recall calculation for actual data
precision, recall, thresholds = precision_recall_curve(Y_test, y_prob_gbo_mtp)
pr_auc = auc(recall, precision)

# Bootstrapping process to calculate 95% CI
for i in range(n_iterations):
    # Resample Y_test and y_prob_gbo_mtp with replacement
    Y_test_resampled, y_prob_resampled = resample(Y_test, y_prob_gbo_mtp)

    # Calculate precision and recall for the bootstrap sample
    precision_resampled, recall_resampled, _ = precision_recall_curve(Y_test_resampled, y_prob_resampled)

    # Calculate AUPRC for the resampled data
    pr_auc_resampled = auc(recall_resampled, precision_resampled)

    # Store the AUPRC score
    auprc_scores.append(pr_auc_resampled)

# Calculate the 95% confidence interval (2.5th and 97.5th percentiles)
lower_bound = np.percentile(auprc_scores, 2.5)
upper_bound = np.percentile(auprc_scores, 97.5)

# Plot the original precision-recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='blue', label=f'PR curve (AUC={pr_auc:.3f})')

# Add labels and title
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='best')
plt.show()

# Print the original AUPRC and 95% CI
print(f'AUPRC: {pr_auc:.3f}')
print(f'95% CI for AUPRC: [{lower_bound:.3f}, {upper_bound:.3f}]')

In [ ]:
##now lets get 95% cI for PR curve for ISS

# Number of bootstrap iterations
n_iterations = 1000
# List to store bootstrapped AUPRC values
auprc_scores = []

# Original precision-recall calculation for actual data
precision, recall, thresholds = precision_recall_curve(Y_test, iss_probs)
pr_auc = auc(recall, precision)

# Bootstrapping process to calculate 95% CI
for i in range(n_iterations):
    # Resample Y_test and y_prob_gbo_mtp with replacement
    Y_test_resampled, y_prob_resampled = resample(Y_test,iss_probs)

    # Calculate precision and recall for the bootstrap sample
    precision_resampled, recall_resampled, _ = precision_recall_curve(Y_test_resampled, y_prob_resampled)

    # Calculate AUPRC for the resampled data
    pr_auc_resampled = auc(recall_resampled, precision_resampled)

    # Store the AUPRC score
    auprc_scores.append(pr_auc_resampled)

# Calculate the 95% confidence interval (2.5th and 97.5th percentiles)
lower_bound = np.percentile(auprc_scores, 2.5)
upper_bound = np.percentile(auprc_scores, 97.5)

# Plot the original precision-recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='blue', label=f'PR curve (AUC={pr_auc:.3f})')

# Add labels and title
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='best')
plt.show()

# Print the original AUPRC and 95% CI
print(f'AUPRC: {pr_auc:.3f}')
print(f'95% CI for AUPRC: [{lower_bound:.3f}, {upper_bound:.3f}]')

In [ ]:
##now lets get 95% cI for PR curve for TRISS

# Number of bootstrap iterations
n_iterations = 1000
# List to store bootstrapped AUPRC values
auprc_scores = []

# Original precision-recall calculation for actual data
precision, recall, thresholds = precision_recall_curve(Y_test, X_TRISS.values)
pr_auc = auc(recall, precision)

# Bootstrapping process to calculate 95% CI
for i in range(n_iterations):
    # Resample Y_test and y_prob_gbo_mtp with replacement
    Y_test_resampled, y_prob_resampled = resample(Y_test,X_TRISS.values)

    # Calculate precision and recall for the bootstrap sample
    precision_resampled, recall_resampled, _ = precision_recall_curve(Y_test_resampled, y_prob_resampled)

    # Calculate AUPRC for the resampled data
    pr_auc_resampled = auc(recall_resampled, precision_resampled)

    # Store the AUPRC score
    auprc_scores.append(pr_auc_resampled)

# Calculate the 95% confidence interval (2.5th and 97.5th percentiles)
lower_bound = np.percentile(auprc_scores, 2.5)
upper_bound = np.percentile(auprc_scores, 97.5)

# Plot the original precision-recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='blue', label=f'PR curve (AUC={pr_auc:.3f})')

# Add labels and title
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='best')
plt.show()

# Print the original AUPRC and 95% CI
print(f'AUPRC: {pr_auc:.3f}')
print(f'95% CI for AUPRC: [{lower_bound:.3f}, {upper_bound:.3f}]')